In [1]:
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath('../'))  # Adjust the path as needed

from my_util import df_to_corr_matrix

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif


from joblib import Parallel, delayed

from pickle import dump , load

In [2]:
TARGET_NUM_OF_FEATURES = 50

In [3]:
# Read data
training_file = "../TrainDataset2024.xls"

data = pd.read_excel(training_file)
data.drop(["ID", "RelapseFreeSurvival (outcome)"], axis=1, inplace=True)
data_no_na = data.replace(999, np.nan)
data_no_na.dropna(ignore_index=True, inplace=True)
X = data_no_na.drop('pCR (outcome)', axis=1)
y = data_no_na['pCR (outcome)']

In [4]:
# Drop highly correlated features
CORR_THRESHOLD = 0.9
# Create a correlation matrix
correlation_matrix = X.corr()

highly_correlated_features = set()

for i in range(len(correlation_matrix.columns)):
  for j in range(i):
    if abs(correlation_matrix.iloc[i, j]) > CORR_THRESHOLD:
        highly_correlated_features.add(correlation_matrix.columns[i])

X_no_highly_correlated = X.drop(columns=highly_correlated_features)

In [5]:
scaler = StandardScaler()
Xs = scaler.fit_transform(X_no_highly_correlated)
Xs = pd.DataFrame(Xs, columns=X_no_highly_correlated.columns)

In [28]:
df = pd.concat([y, Xs], axis=1)

corr = df.corr()[["pCR (outcome)"]]

corr.drop(["pCR (outcome)"], inplace=True)

corr["pCR (outcome)"] = abs(corr["pCR (outcome)"])

sorted = corr.sort_values(by="pCR (outcome)", ascending=False)

sorted

,pCR (outcome)
Gene,0.419255
HER2,0.257349
PgR,0.213667
ER,0.182310
original_firstorder_10Percentile,0.154003
original_ngtdm_Busyness,0.132275
LNStatus,0.128529
original_glrlm_RunLengthNonUniformityNormalized,0.119426
Proliferation,0.116031
TumourStage,0.113840


In [30]:
num_of_features_list = [5, 10, 15, 20, 25, 30, 35]

for n in num_of_features_list:
  with open(f"pkl/corr_{n}_selected_features.pkl", 'wb') as file:
    dump(list(sorted[:n].index), file)
    print(f"Saved {file.name}")



Saved pkl/corr_5_selected_features.pkl
Saved pkl/corr_10_selected_features.pkl
Saved pkl/corr_15_selected_features.pkl
Saved pkl/corr_20_selected_features.pkl
Saved pkl/corr_25_selected_features.pkl
Saved pkl/corr_30_selected_features.pkl
Saved pkl/corr_35_selected_features.pkl
